In [1]:
import requests
import io
import pandas as pd
import regex as re

from bs4 import BeautifulSoup
from pdfminer.high_level import extract_text

from unpy_get_dates import *
from scihub_upgraded import SciHub

from datetime import datetime


In [10]:
import warnings
warnings.filterwarnings("ignore","PDFTextExtractionNotAllowedWarning")


In [2]:
%store -r artcl_df

In [3]:
meta_df = artcl_df.copy()

In [4]:
meta_df_keep = ['doi', 'doi_url', 'title', 'genre','published_date',
       'year', 'journal_name', 'journal_issns', 'journal_issn_l',
       'journal_is_oa', 'journal_is_in_doaj', 'publisher', 'is_oa','oa_status','oa_locations','updated','z_authors']

meta_df_drop = ['is_paratext',  'has_repository_copy', 'best_oa_location',
       'first_oa_location',  'oa_locations_embargoed',
       'data_standard', 'best_oa_location.updated',
       'best_oa_location.url', 'best_oa_location.url_for_pdf',
       'best_oa_location.url_for_landing_page', 'best_oa_location.evidence',
       'best_oa_location.license', 'best_oa_location.version',
       'best_oa_location.host_type', 'best_oa_location.is_best',
       'best_oa_location.pmh_id', 'best_oa_location.endpoint_id',
       'best_oa_location.repository_institution', 'best_oa_location.oa_date',
       'first_oa_location.updated', 'first_oa_location.url',
       'first_oa_location.url_for_pdf',
       'first_oa_location.url_for_landing_page', 'first_oa_location.evidence',
       'first_oa_location.license', 'first_oa_location.version',
       'first_oa_location.host_type', 'first_oa_location.is_best',
       'first_oa_location.pmh_id', 'first_oa_location.endpoint_id',
       'first_oa_location.repository_institution', 'first_oa_location.oa_date']

meta_df.drop(meta_df_drop, axis=1, inplace=True)
meta_df.reset_index(drop=True, inplace=True)

In [5]:
meta_df["Unpy_filter"] = meta_df["oa_locations"].map(Unpy_filterer)

print(f"Unpy (direct) count: {(meta_df['Unpy_filter'].map(type) == list).sum()}")
print(f"Scihub count: {(meta_df['Unpy_filter'].map(type) == str).sum()}")

Unpy (direct) count: 17428
Scihub count: 46691


In [6]:
# Splitting the datasets for easier manipulation

Unpy_mask_v1 = meta_df["Unpy_filter"].map(type) == list
Unpy_df = meta_df[Unpy_mask_v1].copy()


In [7]:
Scihub_mask_v1 = meta_df["Unpy_filter"].map(type) == str
Scihub_df = meta_df[Scihub_mask_v1].copy()

In [64]:
%store Scihub_df

Stored 'Scihub_df' (DataFrame)


### Error & Bug Fixing

In [8]:
# 1000 TEST LOOP:
# 10 artcls per loop -> 100 loops

test_df = Unpy_df[12000:13000].copy()

for k in range(100):
    start = k*10
    end = (k+1)*10 
    doi_slice = test_df[start:end].index.tolist()
    unpy_sess = requests.Session()
    HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'}
    unpy_sess.headers = HEADERS
    now = datetime.now()
    print(f"Starting loop {k+1}, start time: {now}")
    test_df.loc[doi_slice,"Unpy_results"] = test_df.loc[doi_slice,"Unpy_filter"].map(lambda x:Unpy_dates(x,unpy_sess))
    now = datetime.now()
    print(f"{k+1} loop ended, endtime: {now}")

    

Starting loop 1, start time: 2022-01-04 20:05:00.734170
1 loop ended, endtime: 2022-01-04 20:05:57.392691
Starting loop 2, start time: 2022-01-04 20:05:57.402694
2 loop ended, endtime: 2022-01-04 20:06:24.249701
Starting loop 3, start time: 2022-01-04 20:06:24.250251
3 loop ended, endtime: 2022-01-04 20:07:02.446943
Starting loop 4, start time: 2022-01-04 20:07:02.456444
4 loop ended, endtime: 2022-01-04 20:07:29.121433
Starting loop 5, start time: 2022-01-04 20:07:29.126933
5 loop ended, endtime: 2022-01-04 20:08:08.213183
Starting loop 6, start time: 2022-01-04 20:08:08.219691
6 loop ended, endtime: 2022-01-04 20:08:24.501779
Starting loop 7, start time: 2022-01-04 20:08:24.506279
7 loop ended, endtime: 2022-01-04 20:08:49.325006
Starting loop 8, start time: 2022-01-04 20:08:49.325506
8 loop ended, endtime: 2022-01-04 20:09:03.363224
Starting loop 9, start time: 2022-01-04 20:09:03.363724
9 loop ended, endtime: 2022-01-04 20:09:20.140726
Starting loop 10, start time: 2022-01-04 20:09

In [7]:
# The second inf. loop error is on loop 49, so between 480-490 artcls of the testdf
test_df = Unpy_df[12000:13000].copy()
test_df = test_df[480:490]

# index 52626 causes an infinite loop on PDF reading step. No error messages & nothing on the internet. Skipping for now, will deal with it later if necessary
Unpy_df.loc[52626,"Unpy_filter"]

In [10]:
# TESTING CELL!!!
# Has all the possible formats for testing the Unpy_dates process:

unpy_sess = requests.Session()
HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'}
unpy_sess.headers = HEADERS

# Option 1 : Create a test_df 
for ind in test_df.index.tolist():
    print(f"for {ind} starting process...")
    x = test_df.loc[ind,"Unpy_filter"]
    Unpy_dates(x,unpy_sess)

# Option 2: Manual execution of the function for a list of url(s)
req_direct = unpy_sess.get("https://publications.rwth-aachen.de/record/195025/files/195025.pdf", timeout=10)
doi_file = io.BytesIO(req_direct.content)
pdf_read = extract_text(doi_file).replace("\n", "")

# Option 3: Run the Unpy_dates function automatically for a list of url(s)
#Unpy_dates(["https://bmcmedgenomics.biomedcentral.com/track/pdf/10.1186/1755-8794-4-73"],unpy_sess)


### Bug Fix Adjustments

In [9]:
Unpy_df.drop(52626, inplace=True)

### Full Loop of Unpy_df

In [11]:
for k in range(int(Unpy_df.shape[0]/1000)+1):
    start = k*1000
    end = (k+1)*1000 
    if k == 17:
        doi_slice = Unpy_df[start:].index.tolist()
    else:
        doi_slice = Unpy_df[start:end].index.tolist()

    unpy_sess = requests.Session()
    unpy_sess.headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'}

    now = datetime.now()
    print(f"Starting loop {k+1}, start time: {now}")
    Unpy_df.loc[doi_slice,"Unpy_results"] = Unpy_df.loc[doi_slice,"Unpy_filter"].map(lambda x:Unpy_dates(x,unpy_sess))
    
    now = datetime.now()
    print(f"{k+1} loop ended, endtime: {now}")


Starting loop 1, start time: 2022-01-05 12:16:32.972630


C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x0000025215C9AE50> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x0000025215E2EDB0> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


1 loop ended, endtime: 2022-01-05 12:54:46.222898
Starting loop 2, start time: 2022-01-05 12:54:46.234400


C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x0000025218C6EC20> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x0000025212947540> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x0000025217364270> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarn

2 loop ended, endtime: 2022-01-05 13:30:25.812128
Starting loop 3, start time: 2022-01-05 13:30:25.822630


...

16 loop ended, endtime: 2022-01-05 22:29:26.288340
Starting loop 17, start time: 2022-01-05 22:29:26.289840


C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x000002523C0EF090> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x0000025237B3BEF0> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


17 loop ended, endtime: 2022-01-05 23:17:15.467609
Starting loop 18, start time: 2022-01-05 23:17:15.478111


C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x000002523C040130> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)
C:\Users\oguzk\anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BytesIO object at 0x00000252376E7360> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


18 loop ended, endtime: 2022-01-05 23:37:26.913372


### Loop Finished

In [12]:
%store Unpy_df

Stored 'Unpy_df' (DataFrame)


In [45]:
# Analysing the full loop results
Unpy_df.Unpy_results.map(type).value_counts()

<class 'NoneType'>    11461
<class 'list'>         5966
Name: Unpy_results, dtype: int64

In [38]:
# Creating "Unpy_one_fin" to map the loop results & store as a seperate dataframe

Unpy_one_fin = Unpy_df.copy()

Unpy_one_fin.loc[Unpy_one_fin.Unpy_results.map(type)== list,"OK_link"] = Unpy_one_fin.loc[Unpy_one_fin.Unpy_results.map(type)== list,"Unpy_results"].map(lambda x: x[0])
Unpy_one_fin.loc[Unpy_one_fin.Unpy_results.map(type)== list,"Rec_date"] = Unpy_one_fin.loc[Unpy_one_fin.Unpy_results.map(type)== list,"Unpy_results"].map(lambda x: x[1])
Unpy_one_fin.loc[Unpy_one_fin.Unpy_results.map(type)== list,"Acc_date"] = Unpy_one_fin.loc[Unpy_one_fin.Unpy_results.map(type)== list,"Unpy_results"].map(lambda x: x[2])


In [46]:
%store Unpy_one_fin

Stored 'Unpy_one_fin' (DataFrame)
